In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from lightgbm import plot_importance
import shap
import lime
import lightgbm as lgb
from sklearn.ensemble import GradientBoostingClassifier
import json
import pickle
from utils.utils import *
import scipy

C:\Users\thait\anaconda3\envs\thesis\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv("./data/compas/preprocessed.csv", index_col=0)
# df = df.drop_duplicates(subset=df.columns.tolist()[1:], keep='first')
df

,name,sex,age_cat,race,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,is_recid,r_charge_degree,is_violent_recid,vr_charge_degree,score_text,v_decile_score,v_score_text,event
3,kevon dixon,0,0,0,0,3,0,0,0,-1.0,1,7,1,7,0,1,0,1
4,ed philo,0,1,0,0,4,0,1,4,-1.0,1,3,0,0,0,3,0,0
5,ed philo,0,1,0,0,4,0,1,4,-1.0,1,3,0,0,0,3,0,0
6,ed philo,0,1,0,0,4,0,1,4,-1.0,1,3,0,0,0,3,0,0
7,ed philo,0,1,0,0,4,0,1,4,-1.0,1,3,0,0,0,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18310,malcolm simmons,0,1,0,0,3,0,0,0,-1.0,0,0,0,0,0,5,2,0
18311,alexsandra beauchamps,1,0,0,0,6,0,0,5,-1.0,0,0,0,0,2,4,0,0
18313,farrah jean,1,0,0,0,2,0,0,3,-1.0,0,0,0,0,0,2,0,0
18314,florencia sanmartin,1,1,2,0,4,0,0,2,-2.0,1,2,0,0,0,4,0,0


In [3]:
mean_fel = df['juv_fel_count'].mean()
mean_misd = df['juv_misd_count'].mean()
mean_other = df['juv_other_count'].mean()
mean_priors = df['priors_count'].mean()
print(mean_fel, mean_misd, mean_other, mean_priors)

0.08273791654157911 0.10417292104473132 0.1332933053137196 4.186670669468628


In [4]:
data = df.drop(["event", "is_recid"], axis=1)
y = df["is_recid"]
X_train1_temp, X_test1_temp, y_train, y_test = train_test_split(data, y, test_size=0.25, 
                                                                shuffle=True, stratify=y, 
                                                                random_state=2)

In [5]:
cols = data.columns.tolist()
feature_columns = ['name', 'priors_count', 'v_decile_score', 'decile_score',
                   'juv_fel_count', 'juv_misd_count', 'juv_other_count', 
                   'sex', 'age_cat', 'race'] 
        
delete_cols = []
for col in cols:
    if col not in feature_columns:
        delete_cols.append(col)

print(delete_cols)

['days_b_screening_arrest', 'r_charge_degree', 'is_violent_recid', 'vr_charge_degree', 'score_text', 'v_score_text']


In [6]:
X_train1 = X_train1_temp.drop(delete_cols, axis=1)
X_test1 = X_test1_temp.drop(delete_cols, axis=1)
X_train, X_test = X_train1.drop(["name"], axis=1), X_test1.drop(["name"], axis=1)

In [7]:
print(X_test.columns)
feature_names = ['sex', 'age category', 'race', 'juvenile felony count', 'recidivism decile score', 
                 'juvenile misdemeanor count', 'juvenile other count', 'priors count', 
                 'violence decile score']

Index(['sex', 'age_cat', 'race', 'juv_fel_count', 'decile_score',
       'juv_misd_count', 'juv_other_count', 'priors_count', 'v_decile_score'],
      dtype='object')


In [8]:
params = load_dict('./data/compas/parameter.json')
print(params)

{'sex': {'0': 'Male', '1': 'Female'}, 'age_cat': {'0': '25 - 45', '1': 'Less than 25', '2': 'Greater than 45'}, 'race': {'0': 'African-American', '1': 'Caucasian', '2': 'Hispanic', '3': 'Asian', '4': 'Native American'}, 'score_text': {'0': 'Low', '1': 'High', '2': 'Medium'}, 'v_score_text': {'0': 'Low', '1': 'High', '2': 'Medium'}, 'names': {'sex': 'sex', 'age_cat': 'age category', 'race': 'race', 'juv_fel_count': 'juvenile felony count', 'juv_misd_count': 'juvenile misdemeanor count', 'juv_other_count': 'juvenile other count', 'priors_count': 'priors count', 'decile_score': 'recidivism decile score', 'v_decile_score': 'violent decile score'}}


# ------------------------------------------------------------------------------------------

In [9]:
# gbtree_d = GradientBoostingClassifier(learning_rate=0.10, n_estimators=116, max_depth=3, subsample=0.74)
# gbtree_d.fit(X_train.values, y_train.values)
# gbtree_d.score(X_test.values, y_test.values)

In [10]:
# # save model
# pickle.dump(gbtree_d, open("./model/unbiased_model.pickle.dat", "wb"))
gbtree_d = pickle.load(open("./model/Model-Desired.pickle.dat", "rb"))
gbtree_d.score(X_test.values, y_test.values)

0.6671469740634006

In [11]:
# explainer_d = shap.Explainer(gbtree_d)
# shap_values_d = explainer_d(X_test.values)
# shap.summary_plot(shap_values_d.values, X_test.values, 
#                   feature_names=feature_names, plot_type="dot", show=True)
# # plt.savefig("./model/explanation/SHAP/global_explanation_beeswarm_desired.pdf", bbox_inches='tight')
# # plt.savefig("./model/explanation/SHAP/global_explanation_beeswarm_desired.png")#, bbox_inches='tight')
# plt.clf()

In [12]:
# noise = np.random.normal(10, 15, X_train.values.shape)
# noise[:, :3] = 0
# # print(noise)
# # X_train#.values#[0] + noise[0]

In [13]:
# gbtree_p = GradientBoostingClassifier(learning_rate=0.10, n_estimators=116, max_depth=3, subsample=0.74)
# gbtree_p.fit(X_train.values + noise, y_train.values)
# gbtree_p.score(X_test.values, y_test.values)

In [14]:
# save model
# pickle.dump(gbtree_p, open("./model/biased_model.pickle.dat", "wb"))
gbtree_p = pickle.load(open("./model/Model-Prohibited.pickle.dat", "rb"))
gbtree_p.score(X_test.values, y_test.values)

0.5823727185398655

In [15]:
# explainer_p = shap.Explainer(gbtree_p)
# shap_values_p = explainer_p(X_test.values)
# shap.summary_plot(shap_values_p.values, X_test.values, 
#                   feature_names=feature_names, plot_type="dot", show=True)
# # plt.savefig("./model/explanation/SHAP/global_explanation_beeswarm_prohibited.pdf", bbox_inches='tight')
# # plt.savefig("./model/explanation/SHAP/global_explanation_beeswarm_prohibited.png")#, bbox_inches='tight')
# plt.clf()

# ------------------------------------------------------------------------------------------

In [16]:
person_ids_p = gbtree_p.predict_proba(X_test.values)[:, 0].argsort()
person_ids_p

array([1163,  817, 1955, ..., 2514, 2707, 1653], dtype=int64)

In [17]:
from tqdm import tqdm
idxs = []
for i in tqdm(person_ids_p):
    pred_prohibited = gbtree_p.predict(X_test.values)[i]
    pred_desired = gbtree_d.predict(X_test.values)[i]
    if pred_prohibited != pred_desired:
        idxs.append(i)

100%|█████████████████████████████████████████████████████████████████████████████| 4164/4164 [00:38<00:00, 106.85it/s]


In [18]:
print(len(idxs))

1269


In [19]:
race_0 = []
race_1 = []
race_2 = []
race_3 = []
race_4 = []
for i in idxs:
    if X_test1.iloc[i].race == 0:
        race_0.append(i)
    elif X_test1.iloc[i].race == 1:
        race_1.append(i)
    elif X_test1.iloc[i].race == 2:
        race_2.append(i)
    elif X_test1.iloc[i].race == 3:
        race_3.append(i)
    elif X_test1.iloc[i].race == 4:
        race_4.append(i)
# [(0, 'African-American'), (1, 'Caucasian'), (2, 'Hispanic'), (3, 'Asian'), (4, 'Native American')]
print("race 0 (Affrican-America):", len(race_0))
print("race 1 (Caucasian):", len(race_1))
print("race 2 (Hispanic):", len(race_2))
print("race 3 (Asian):", len(race_3))
print("race 4 (Native American):", len(race_4))

race 0 (Affrican-America): 699
race 1 (Caucasian): 463
race 2 (Hispanic): 95
race 3 (Asian): 5
race 4 (Native American): 7


In [20]:
prob_race0 = gbtree_p.predict_proba(X_test.values)[race_0]
i_race0 = [x for _, x in sorted(zip(prob_race0[:, 0], race_0))]

prob_race1 = gbtree_p.predict_proba(X_test.values)[race_1]
i_race1 = [x for _, x in sorted(zip(prob_race1[:, 0], race_1))]

prob_race2 = gbtree_p.predict_proba(X_test.values)[race_2]
i_race2 = [x for _, x in sorted(zip(prob_race2[:, 0], race_2))]

prob_race3 = gbtree_p.predict_proba(X_test.values)[race_3]
i_race3 = [x for _, x in sorted(zip(prob_race3[:, 0], race_3))]

prob_race4 = gbtree_p.predict_proba(X_test.values)[race_4]
i_race4 = [x for _, x in sorted(zip(prob_race4[:, 0], race_4))]

# -----------------------------------------------------------------------------------------

In [21]:
print(i_race4, len(race_4))

[1340, 1921, 2242, 120, 1457, 1654, 1914] 7


In [22]:
np.random.seed(2)

# rand_ids = np.concatenate((np.random.choice(i_race0, size=40), np.random.choice(i_race1, size=20)))
# print(rand_ids)
rand_race0 = np.random.choice(i_race0, size=200, replace=False)
print(rand_race0)
rand_race1 = np.random.choice(i_race1, size=200, replace=False)
rand_race2 = np.random.choice(i_race2, size=len(race_2), replace=False)
rand_race3 = np.random.choice(i_race3, size=len(race_3), replace=False)
rand_race4 = np.random.choice(i_race4, size=len(race_4), replace=False)
path = './model/explanation/get_prediction'
features = ['desired_features', 'prohibited_features']
plot_types = ['bar']
races = [rand_race0, rand_race1, rand_race2, rand_race3, rand_race4]

[1266 1170 3476 2878 2116 2267 1101 1758 1686 2030 3916 3498  904 3434
 3579 2464 2394 2024 3103 2292  152 2963 2844 1001  134 3827 1087  113
 1624 2181 2805 2387 2662  467 3873  327 4037 1688 1420 3491 3232 2333
  254  694 2265 2184 1520 1506  502   91 2986 2269  158 2227 1502  998
 1882 2020 3084 1754 3530 3560 1514 2880 2370 2105 2183  716 1696 2505
 1080  744 2927 1741  344  724  260 1549 2335 1300 3751 1927 2070 3394
 2716 2571 1371 3668 1711  747 2929 1026  449 2219 4109  218 1278 2951
 1878 2136 3182 3438 3798  874 2576 1485 1236 3160 3379 2166 1214 4050
 1106 2801 2192 3779 3809 2800 3109 2861 1329   48 3343 3957 1252  985
 2562 2917  951 2390 1994 1691 2886 3585 1161 1594   26 1145 2600 3501
  823 2355  642  494  903 1271 2714 2472 3601 1518 1137  422 1023  691
 1125 1127 1169 3656  718 1622  510  908 1729  538  328 3666 1587  557
  486 3612 1173  573 3344 3213  890  156   64 2397 1630 3133 4084  160
 1072 2211  715   47 1093  938 2323 2683 2594 2975  614 1021  987  775
 2029 

In [23]:
# print(X_test.iloc[1266])
# print(y_test.iloc[1266])
# print(gbtree_d.predict(X_test.values)[1266])

In [24]:
# print(X_test.loc[9126])
# print(y_test.loc[9126])
# print(gbtree_d.predict(X_test.values)[1266])

In [25]:
# for plot_type in plot_types:
#     for feature in features: 
#         sv_list = []
#         for i, race in enumerate(races):
#             save_path = os.path.join(path, feature, f"{i}")
#             os.makedirs(save_path, exist_ok=True)
#             for idx in tqdm(race):
#                 if feature == 'desired_features':
#                     tree = gbtree_d
#                 else:
#                     tree = gbtree_p
#                 pred = tree.predict(X_test.values)
#                 name = f"{X_test.iloc[idx].name}_{pred[idx]}"
#                 sv = get_shap_explanation(tree, X_test, params, idx)
#                 if any(obj.data == sv.data for obj in sv_list):
#                     continue
#                 else:
#                     sv_list.append(sv)
                    
#                     plot_shap_explanation(sv, plot_type=plot_type, save=True, save_path=save_path, save_name=name)
#                     plot_shap_explanation(sv, plot_type=plot_type, save=True, save_path=save_path, save_name=name,
#                                          file_type="png")

In [26]:
# select_idxs = [87,125,147,449,566,608,658,660,686,1286,1442,1584,1725,1794,1819,1828,2023,2207,2309,2966,3159,3789,
#                3808,4180,4630,4744,4748,4799,5118,5183,5226,5238,5368,5705,5941,6031,6692,7865,10062,10335,11937,
#                13285,13460,14149,16772,16947,17276,17313,17491,17760]
# print(len(select_idxs))
# path = './model/explanation/get_prediction'
# for plot_type in plot_types:
#     for feature in features: 
#         sv_list = []
#         for i, race in enumerate(races):
#             save_path = os.path.join(path, feature, f"{i}")
#             os.makedirs(save_path, exist_ok=True)
#             for idx in tqdm(race):
#                 if X_test.iloc[idx].name not in select_idxs:
#                     continue
#                 if feature == 'desired_features':
#                     tree = gbtree_d
#                 else:
#                     tree = gbtree_p
#                 pred = tree.predict(X_test.values)
#                 ground_truth = y_test.iloc[idx]
#                 name = f"{X_test.iloc[idx].name}_{pred[idx]}_truth{ground_truth}"
#                 sv = get_shap_explanation(tree, X_test, params, idx)
#                 if any(obj.data == sv.data for obj in sv_list):
#                     continue
#                 else:
#                     sv_list.append(sv)
                    
#                     plot_shap_explanation(sv, plot_type=plot_type, save=True, save_path=save_path, save_name=name)
#                     plot_shap_explanation(sv, plot_type=plot_type, save=True, save_path=save_path, save_name=name,
#                                          file_type="png")